In [26]:
import boto3
from boto.s3.connection import S3Connection
import boto
from boto.s3.key import Key
import sys
import os
import json
import pandas as pd
from collections import defaultdict
from matplotlib import pyplot as plt
import ast 
from utils.credentials import access_key, secret_access_key
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
import sys
import os
sys.path.insert(0,os.path.abspath('..'))

In [63]:
import utils.preprocessing as prep
import utils.ldamodel as lda
import imp
imp.reload(lda)

<module 'utils.ldamodel' from '/Users/miraekim/workspace/coursework/persproj/utils/ldamodel.py'>

In [27]:
client = boto3.client('s3')

conn = boto.s3.connect_to_region('us-east-2',
       aws_access_key_id=access_key,
       aws_secret_access_key=secret_access_key,
       is_secure=True,
       calling_format = boto.s3.connection.OrdinaryCallingFormat(),
       )

bucket = conn.get_bucket('persproj')

In [28]:
%%time
food_review_chunks = pd.read_csv('s3://persproj/data/food_review_df.csv', chunksize=1000, iterator=True)

CPU times: user 143 ms, sys: 64.7 ms, total: 208 ms
Wall time: 2.33 s


In [29]:
food_review_df = pd.concat(food_review_chunks, ignore_index=True)

In [57]:
bow_corpus = np.load('../models/2020_03_03/bow.npy')

In [39]:
# load the indices of the dataframe used 

file = open('../models/2020_03_03/df_index.txt', 'r')
file_text = file.read()
df_index = [int(x) for x in file_text.split(' ')]

In [43]:
food_review_df_sample = food_review_df.iloc[df_index]

In [68]:
from gensim.models.ldamodel import LdaModel
from gensim.corpora import Dictionary

lda = LdaModel.load('../models/2020_03_03/model_2020_03_02_01perc.model')

loaded_dct = Dictionary.load_from_text('../models/2020_03_03/dictionary.txt')

In [78]:
wordnet

<WordNetCorpusReader in '/Users/miraekim/nltk_data/corpora/wordnet'>

In [79]:
from nltk.corpus import wordnet

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(text, pos=get_wordnet_pos(text))


def stem(text):
    stemmer = SnowballStemmer("english", ignore_stopwords=True)
    return stemmer.stem(text)

def get_bigrams(docs):

    # Add bigrams and trigrams to docs (only ones that appear 20 times or more).
    bigram = Phrases(docs, min_count=20)
    for idx in range(len(docs)):
        for token in bigram[docs[idx]]:
            if '_' in token:
                # Token is a bigram, add to document.
                docs[idx].append(token)
    return docs

def preprocess_words(text):

    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(stem(lemmatize(token)))

    return result

In [82]:
unseen_text = '''
One of the best Korean hot pots I've had here in San Francisco! Not too long of a wait considering a weekend night, this little joint is always packed to the brim so be sure to sign in on yelp to reserve your spot!

We were shown the menu and taken orders before seating so the food came immediately as we sat down which was so convenient. The venue was small and intimate but enough room to fit large parties as most dishes were meant for big groups.

'''

unseen_text_corpus = preprocess_words(unseen_text)

In [84]:
one_doc = loaded_dct.doc2bow(unseen_text_corpus)

In [85]:
lda.get_document_topics(one_doc)

[(0, 0.047711503), (2, 0.30772126), (4, 0.4017404), (5, 0.233078)]

In [87]:
lda.print_topics()

[(0,
  '0.020*"good" + 0.020*"great" + 0.017*"servic" + 0.016*"order" + 0.014*"time" + 0.014*"come" + 0.011*"burger" + 0.011*"like" + 0.010*"friend" + 0.010*"fri"'),
 (1,
  '0.013*"great" + 0.013*"good" + 0.012*"time" + 0.010*"restaur" + 0.009*"like" + 0.008*"night" + 0.008*"go" + 0.008*"come" + 0.007*"love" + 0.007*"best"'),
 (2,
  '0.016*"order" + 0.012*"time" + 0.012*"drink" + 0.012*"come" + 0.010*"servic" + 0.010*"like" + 0.009*"wait" + 0.009*"go" + 0.009*"tabl" + 0.008*"say"'),
 (3,
  '0.020*"good" + 0.014*"pizza" + 0.014*"order" + 0.012*"chicken" + 0.012*"like" + 0.009*"sauc" + 0.008*"taco" + 0.008*"come" + 0.008*"great" + 0.007*"tast"'),
 (4,
  '0.011*"like" + 0.009*"room" + 0.008*"good" + 0.008*"time" + 0.007*"great" + 0.007*"cream" + 0.006*"look" + 0.006*"nice" + 0.006*"store" + 0.006*"friend"'),
 (5,
  '0.012*"good" + 0.011*"restaur" + 0.009*"great" + 0.008*"servic" + 0.008*"dish" + 0.007*"come" + 0.007*"order" + 0.007*"menu" + 0.006*"delici" + 0.006*"perfect"')]

In [58]:
def analyze_review(review_id,df):
    ind = list(df.review_id.values).index(review_id)
    print('Review')
    print(df['text'].iloc[ind])
    print('\n')
    print('Corpus')
    print(sorted([(lda_class.dictionary.get(x[0]), x[1]) for x in lda_class.bow_corpus[ind]], key=lambda x: x[1], reverse=True))
    print('\n')
    print('Topics')
    print(sorted(lda.get_document_topics(lda_class.bow_corpus[ind]), key=lambda x: x[1], reverse=True))
    
def get_topic_dist(review_id):
    ind = list(food_review_df_sample.review_id.values).index(review_id)
    return dict(lda.get_document_topics(bow_corpus[ind]))

In [67]:
lda.models.LdaMulticore.get_topics()

TypeError: get_topics() missing 1 required positional argument: 'self'

In [55]:
lda.get_document_topics()

In [59]:
food_review_df_sample[:10].apply(lambda x: get_topic_dist(x['review_id']),1)
# food_review_df_sample['topic_dist'] = food_review_df_sample.apply(lambda x: get_topic_dist(x['review_id']),1)

3904692         {0: 0.013375101, 2: 0.3011184, 3: 0.6690545}
2611438         {0: 0.36692908, 2: 0.06574727, 4: 0.5581931}
4531851       {0: 0.71988994, 1: 0.012711138, 2: 0.24739583}
822534     {0: 0.27208763, 1: 0.51034594, 2: 0.100129515,...
2949890                       {0: 0.36702406, 1: 0.61500376}
972433         {1: 0.26047656, 2: 0.01890594, 3: 0.71009094}
2413348       {0: 0.012311184, 2: 0.72052485, 3: 0.25155514}
38666          {0: 0.020279497, 1: 0.01281158, 4: 0.9438786}
4312393    {0: 0.12711872, 1: 0.27447778, 2: 0.19878481, ...
1143306          {2: 0.629739, 3: 0.2738738, 4: 0.076821215}
dtype: object

In [46]:
from surprise import Dataset
from surprise import Reader
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(food_review_df_sample[['city','relevant_topic', 'review_stars']], reader)

KeyError: "['relevant_topic'] not in index"

In [ ]:
# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)